# Conducción de Calor en 2D

**Objetivo General**
- Resolver numérica y computacionalmente la ecuación de conducción de calor en dos dimensiones.

**Objetivos particulares**
- Definir los parámetros físicos y numéricos.
- Definir la malla del dominio.
- Definir la temperatura inicial junto con sus condiciones de frontera y graficarla sobre la malla.
- Definir el sistema lineal y resolverlo.
- Graficar la solución.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import FD2D as fd2
#import macti.visual as mvis

# TODO: hacer este cambio en macti.visual
plt.rcParams['axes.grid'] = False

## Flujo de calor

Fourier también estableció una ley para el flujo de calor que se escribe como:

$$
\vec{q} = -\kappa \nabla T = -\left(\kappa _x \dfrac{\partial T}{\partial x}, \kappa _y \dfrac{\partial T}{\partial y}\right)
$$

Usando la información calculada de la temperatura (almacenada en el arreglo `u`), vamos a calcular el flujo de calor usando la siguiente fórmula en diferencias:

$$
\vec{q}_{i,j} = \big(qx_{i,j}, qy_{i,j}\big) = - \Big(\dfrac{\kappa_x}{2h_x} (T_{i+1,j}-T_{i-1,j}), \dfrac{\kappa_y}{2h_y} (T_{i,j+1}-T_{i,j-1}) \Big)
$$

In [2]:
def calc_heat_flux(T, hx, hy, kx, ky):
    qx = np.zeros(T.shape)
    qy = np.zeros(T.shape)

    print(qx.shape, qy.shape)
    
    NyT, NxT = T.shape

    sx = kx / 2*hx
    sy = ky / 2*hy
    
    for i in range(1,NyT-1):
        for j in range(1,NxT-1):
            qx[i,j] = -sx * (T[i+1,j] - T[i-1,j])
            qy[i,j] = -sy * (T[i,j+1] - T[i,j-1])
            
    return qy, qx

## Parámetros físicos y numéricos

In [3]:
def conduccion2D_i(Lx, Ly, kx, ky, TC, TH, F1, F2, S, IS, JS, Nx, Ny, grad):
    # Número total de nodos en cada eje incluyendo las fronteras
    NxT = Nx + 2
    NyT = Ny + 2

    # Número total de nodos
    NT = NxT * NyT

    # Número total de incógnitas
    N = Nx * Ny

    # Tamaño de la malla en cada dirección
    hx = Lx / (Nx+1)
    hy = Ly / (Ny+1)

    # Coordenadas de la malla
    xn = np.linspace(0,Lx,NxT)
    yn = np.linspace(0,Ly,NyT)

    # Generación de una rejilla
    xg, yg = np.meshgrid(xn, yn, indexing='ij')

    # Definición de un campo escalar en cada punto de la malla
    T = np.zeros((NyT, NxT))

    # Calculo de los coeficientes para la matriz
    a, b, c = fd2.calc_coef_FDM(kx, ky, hx, hy)

    if grad == 'vertical':
        FL = F1
        FR = F2
        T0 = fd2.calc_grad_vertical(T, TH, TC, FL, FR, S, IS, JS, hx, a, b, c)
    elif grad == 'horizontal':
        FB = F1
        FT = F2
        T0 = fd2.calc_grad_horizontal(T, TH, TC, FB, FT, S, IS, JS, hy, a, b, c)
    
    qx, qy = calc_heat_flux(T, hx, hy, kx, ky)
    
    fd2.plot_solution(T, T0, qx, qy, Lx, Ly, xg, yg, xn, yn)

In [4]:
w_Lx = widgets.BoundedFloatText(value=1.0, min = 1.0, description='Lx', disabled=False, layout=widgets.Layout(width='200px'))
w_Ly = widgets.BoundedFloatText(value=1.0, min = 1.0, description='Ly', disabled=False, layout=widgets.Layout(width='200px'))
w_kx = widgets.BoundedFloatText(value=1.0, min = 1.0, description='$\kappa_x$', disabled=False, layout=widgets.Layout(width='200px'))
w_ky = widgets.BoundedFloatText(value=1.0, min = 1.0, description='$\kappa_y$', disabled=False, layout=widgets.Layout(width='200px'))
w_TC = widgets.FloatText(value=-1.0, description='TC', disabled=False, layout=widgets.Layout(width='200px'))
w_TH = widgets.FloatText(value= 1.0, description='TH', disabled=False, layout=widgets.Layout(width='200px'))
w_F1 = widgets.FloatText(value=0.0, description='F1', disabled=False, layout=widgets.Layout(width='200px'))
w_F2 = widgets.FloatText(value=0.0, description='F2', disabled=False, layout=widgets.Layout(width='200px'))

w_S  = widgets.FloatText(value=0.0, description='S', disabled=False, 
                         layout=widgets.Layout(width='200px'))

w_IS = widgets.IntSlider(value=1, min=1, max=40, step=1,
                         description='IS', disabled=False, 
                         style=dict(handle_color = '#CD5C5C'),
                         layout=widgets.Layout(width='250px'))
w_JS = widgets.IntSlider(value=1, min=1, max=40, step=1,
                         description='JS', disabled=False, 
                         style=dict(handle_color = '#CD5C5C'),
                         layout=widgets.Layout(width='250px'))
w_Nx = widgets.IntSlider(value=4, min=4, max=40, step=2,
                         description='Nx', disabled=False, 
                         style=dict(handle_color = '#3CB371'),
                         layout=widgets.Layout(width='250px'))
w_Ny = widgets.IntSlider(value=4, min=4, max=40, step=2,
                         description='Ny', disabled=False, 
                         style=dict(handle_color = '#3CB371'),
                         layout=widgets.Layout(width='250px'))

restrict_IS = widgets.link(
    (w_Nx, 'value'),  # Valor del primer slider ligado con
    (w_IS, 'max')     # el máximo del segundo slider
)

restrict_JS = widgets.link(
    (w_Ny, 'value'),  # Valor del primer slider ligado con
    (w_JS, 'max')     # el máximo del segundo slider
)

w_grad = widgets.SelectionSlider(value = 'vertical', options=['vertical','horizontal'], 
                              description='Gradiente',
                              layout=widgets.Layout(width='200px'))

button = widgets.Button(description="Run", icon='play', 
                        style=dict(button_color='lightgreen'),
                        layout=widgets.Layout(width='100px')    
)
title = widgets.HTML(value=r"Simulación de la conducción de calor en un infierno en 2D rectangular usando Diferencias Finitas")
eq = widgets.HTMLMath(value=r"$$-\left(\kappa_x\frac{\partial^2 T}{\partial x^2} + \kappa_y\frac{\partial^2 T}{\partial y^2}\right)= S$$",
description='')

ti = widgets.VBox([title, eq], 
                  layout=widgets.Layout(border = 'solid 1px black', 
                                        width='700px',
                                        align_items='center'))
ui = widgets.HBox([widgets.VBox([w_Nx, w_Ny, w_IS, w_JS, w_S]),
                   widgets.VBox([w_Lx, w_Ly, w_kx, w_ky, w_grad]),
                   widgets.VBox([w_TC, w_TH, w_F1, w_F2, button],
                               layout=widgets.Layout(align_items = 'flex-end')),
                  ])

ui.layout = widgets.Layout(border='solid 1px black')
ui.layout.width = '700px'

output = widgets.Output()

display(ti, ui, output)

def on_button_clicked(b):
    output.clear_output(wait=True)
    with output:
        conduccion2D_i(Lx = w_Lx.value, Ly = w_Ly.value, 
                       kx = w_kx.value, ky = w_ky.value,
                       TC = w_TC.value, TH = w_TH.value,
                       Nx = w_Nx.value, Ny = w_Ny.value,
                       S = w_S.value,
                       IS = w_IS.value, JS = w_JS.value,
                       F1 = w_F1.value, F2 = w_F2.value,
                       grad = w_grad.value) 
button.on_click(on_button_clicked)

Output()